<a href="https://colab.research.google.com/github/Swayamprakashpatel/HPLC/blob/main/HPLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pandas_datareader as web
!pip install pubchempy
import pubchempy as pcp
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from google.colab import files
import time as tm

In [ ]:
df1 = pd.read_csv('/content/Data.csv')
df1 = pd.DataFrame(df1)
df1 = df1.iloc[:,0:25]

cidvals = df1['CID_1']
componentFingerprint = []
for cid in cidvals:
  c = pcp.Compound.from_cid(cid)
  componentFingerprint.append(c.cactvs_fingerprint)
  tm.sleep(0.1)

df2 = df1.assign(fp=componentFingerprint)
i = 1
for  componentFingerprint in c.cactvs_fingerprint:
    df2['fp'] = df2['fp'].astype(str)
    df2['fp'+str (i)] = df2['fp'].str[i-1:i]
    i = i+ 1
#print(componentFingerprint)
#print(df2)

#df2.to_csv('Drug_1.csv')

#FOR DRUGS_2

cidvals = df1['CID_2']
componentFingerprint = []
for cid in cidvals:
  c = pcp.Compound.from_cid(cid)
  componentFingerprint.append(c.cactvs_fingerprint)
  tm.sleep(0.1)

df3 = df1.assign(fp=componentFingerprint)
i = 1
for  componentFingerprint in c.cactvs_fingerprint:
    df3['fp'] = df3['fp'].astype(str)
    df3['fp'+str (i)] = df3['fp'].str[i-1:i]
    i = i+ 1
#print(componentFingerprint)
#print(df2)

#df3.to_csv('Drug_1.csv')


#INPUT DATA
df2 = df2.iloc[:,23:904]
df3 = df3.iloc[:,23:904]
X = pd.concat([df2,df3], axis = 1)
print(X,X.shape)
X.head()



#Output Data
Y1 = df1.iloc[:,4:11]
Y1.head()

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(Y1)
OneHotEncoder(handle_unknown='ignore')
enc.categories_

Y2 = enc.transform(Y1).toarray() #Categorical Data
print(Y2,Y2.shape)

Y3 = df1.iloc[:,11:22] #Numerical Data



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


     fp1 fp2 fp3 fp4 fp5 fp6 fp7 fp8 fp9 fp10  ... fp872 fp873 fp874 fp875  \
0      1   1   0   0   0   0   0   0   0    1  ...     0     0     0     0   
1      1   1   0   0   0   0   0   0   0    1  ...     0     0     0     0   
2      1   1   0   0   0   0   0   0   0    1  ...     0     0     0     0   
3      1   1   0   0   0   0   0   0   0    1  ...     0     0     0     0   
4      1   1   0   0   0   0   0   0   0    1  ...     0     0     0     0   
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...   ...   ...   ...   ...   
1103   1   1   1   0   0   0   0   0   0    1  ...     0     0     0     0   
1104   1   1   0   0   0   0   0   0   0    1  ...     0     0     0     0   
1105   1   1   1   1   0   0   0   0   0    1  ...     0     0     0     0   
1106   1   1   1   1   0   0   0   0   0    1  ...     0     0     0     0   
1107   1   0   0   0   0   0   0   0   0    1  ...     0     0     0     0   

     fp876 fp877 fp878 fp879 fp880 fp881  
0        0     0    

In [ ]:
df1 = pd.read_csv('/content/Data.csv')
df1 = pd.DataFrame(df1)
#df1 = df1.iloc[:,0:25]

X = pd.read_csv('/content/CID.csv')
X = pd.DataFrame(X)
X = X.iloc[:,:]


Y1 = df1.iloc[:,4:11]
Y1.head()

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(Y1)
OneHotEncoder(handle_unknown='ignore')
enc.categories_

Y2 = enc.transform(Y1).toarray() #Categorical Data
print(Y2,Y2.shape)

Y3 = df1.iloc[:,11:22] #Numerical Data


[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]] (1108, 164)


In [ ]:
print(X,X.shape)
X.head()
Y1.head()
print(Y2,Y2.shape)


      fp1  fp2  fp3  fp4  fp5  fp6  fp7  fp8  fp9  fp10  ...  fp872.1  \
0       1    1    0    0    0    0    0    0    0     1  ...        0   
1       1    1    0    0    0    0    0    0    0     1  ...        0   
2       1    1    0    0    0    0    0    0    0     1  ...        0   
3       1    1    0    0    0    0    0    0    0     1  ...        0   
4       1    1    0    0    0    0    0    0    0     1  ...        0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  ...      ...   
1103    1    1    1    0    0    0    0    0    0     1  ...        0   
1104    1    1    0    0    0    0    0    0    0     1  ...        0   
1105    1    1    1    1    0    0    0    0    0     1  ...        0   
1106    1    1    1    1    0    0    0    0    0     1  ...        0   
1107    1    0    0    0    0    0    0    0    0     1  ...        0   

      fp873.1  fp874.1  fp875.1  fp876.1  fp877.1  fp878.1  fp879.1  fp880.1  \
0           0        0        0        0   

In [ ]:
print(X.shape)

(1108, 1762)


In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

X_train, X_val_and_test, Y2_train, Y2_val_and_test = train_test_split(X, Y2, test_size=0.3,random_state = 42 )
X_val, X_test, Y2_val, Y2_test = train_test_split(X_val_and_test, Y2_val_and_test, test_size=0.5, random_state= 42)

X_train = np.asarray(X_train).astype(np.int64)
X_val = np.asarray(X_val).astype(np.int64)
X_test = np.asarray(X_test).astype(np.int64)
Y2_train = np.asarray(Y2_train).astype(np.int64)
Y2_val = np.asarray(Y2_val).astype(np.int64)
Y2_test = np.asarray(Y2_test).astype(np.int64)

In [ ]:
filepath = '/content/drive/MyDrive/Model_DE/Model.hdf5'
 
checkpoint = [tf.keras.callbacks.ModelCheckpoint(filepath, monitor='accuracy', mode='auto', save_best_only=True, Save_weights_only = False, verbose = 1), 
              tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=25, verbose =2)]

output_nodes = Y2.shape[1]
print(output_nodes)
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(3524, activation='relu', input_shape=(1762,)),
                             tf.keras.layers.Dense(1512, activation='relu'),
                             tf.keras.layers.Dense(1512, activation='relu'),
                             tf.keras.layers.Dense(output_nodes, activation= 'softmax')])
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss=tf.keras.losses.BinaryCrossentropy(from_logits = False), metrics=['accuracy'])
hist = model.fit(X_train, Y2_train, epochs= 2000, callbacks=[checkpoint],validation_data=(X_val, Y2_val), batch_size= 500)
model.evaluate(X_test, Y2_test)
 
Y2_train_predict = np.round(model.predict(X_train))
Y2_train_l = tf.argmax(Y2_train, axis = 1)
Y2_train_predict_l = tf.argmax(Y2_train_predict, axis =1)
import sklearn.metrics as skm
cm = skm.multilabel_confusion_matrix(Y2_train_l, Y2_train_predict_l)
print(cm)
print( skm.classification_report(Y2_train_l, Y2_train_predict_l))
 
train_acc = max(hist.history['accuracy'])
val_acc = max(hist.history['val_accuracy'])
train_loss = min(hist.history['loss'])
val_loss = min(hist.history['val_loss'])
print('Training Accuracy is')
print(train_acc)
print('Validation Accuracy is')
print(val_acc)
print('Training loss is')
print(train_loss)
print('Validation loss is')
print(val_loss)

164
Epoch 1/2000
2/2 [==============================] - ETA: 0s - loss: 0.6763 - accuracy: 0.0000e+00
Epoch 1: accuracy improved from -inf to 0.00000, saving model to /content/drive/MyDrive/Model_DE/Model.hdf5
2/2 [==============================] - 2s 907ms/step - loss: 0.6763 - accuracy: 0.0000e+00 - val_loss: 0.3947 - val_accuracy: 0.4458
Epoch 2/2000
2/2 [==============================] - ETA: 0s - loss: 0.3416 - accuracy: 0.4529
Epoch 2: accuracy improved from 0.00000 to 0.45290, saving model to /content/drive/MyDrive/Model_DE/Model.hdf5
2/2 [==============================] - 1s 763ms/step - loss: 0.3416 - accuracy: 0.4529 - val_loss: 0.3873 - val_accuracy: 0.4458
Epoch 3/2000
2/2 [==============================] - ETA: 0s - loss: 0.2781 - accuracy: 0.2929
Epoch 3: accuracy did not improve from 0.45290
2/2 [==============================] - 1s 522ms/step - loss: 0.2781 - accuracy: 0.2929 - val_loss: 0.0746 - val_accuracy: 0.0000e+00
Epoch 4/2000
2/2 [==============================]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
t1 = model.predict(X_train[0:5,:])
print(t1)
 
t = np.round(model.predict(X_train))
print(t)
 
Y_prediction = enc.inverse_transform(t)
print(Y1.head(),Y_prediction, Y_prediction.shape)


[[1.71530455e-01 8.16830218e-01 6.65873110e-01 3.60201508e-01
  1.99705362e-04 2.29805708e-04 6.58282876e-01 3.73521060e-01
  2.51263380e-04 2.94816494e-03 1.98728012e-05 8.19228371e-05
  3.87079358e-01 4.11924448e-05 2.22253799e-03 4.13147867e-01
  2.75138021e-03 2.40655124e-01 8.58038664e-04 1.06510520e-03
  6.77317381e-04 7.55854368e-01 2.60028243e-03 2.05155629e-05
  2.05650061e-01 1.74760818e-04 9.11428579e-05 9.99983191e-01
  7.51295805e-01 2.40557075e-01 8.49283970e-05 2.69323587e-04
  1.92046165e-04 1.62856122e-05 1.26387904e-05]
 [4.54753081e-06 9.99998808e-01 2.39379663e-08 9.99999285e-01
  1.53970058e-21 4.55224325e-17 9.99996603e-01 1.51708662e-06
  5.24412413e-15 5.02116237e-10 1.25044635e-20 1.46101262e-18
  9.99990106e-01 7.79245744e-18 7.07244983e-12 3.66711191e-07
  1.82847085e-10 8.12716561e-09 7.39361304e-14 1.21551515e-14
  1.19079825e-18 2.98875129e-05 3.23215318e-05 9.99941826e-01
  7.37296613e-11 6.56344163e-12 6.15678307e-15 1.00000000e+00
  1.00000000e+00 1.538

In [ ]:
X.to_csv('Drug_1.csv')